In [1]:
# Imports
import torch
import os 
import sys
from dask_jobqueue import SLURMCluster
from distributed import Client
from pathlib import Path

import do_ablation_impspec as file

In [2]:
# Args setup
ntrial = 50
n = 100
ntest = 100
d = 5
noise = 0.5
niter = 1000
calibrate = [False, True, True]
sample_split = [False, False, True]
marginal_loss = [False, False, False]
retrain_hypers = [False, False, False]
kernel = "gaussian"

In [3]:
# Cluster creation
cluster = SLURMCluster(
    n_workers=0,
    memory="32GB",
    processes=1,
    cores=1,
    scheduler_options={
        "dashboard_address": ":11111",
        "allowed_failures": 10
    },
    job_cpu=1,
    walltime="3:0:0",
    
    job_extra_directives = ["-p medium,fast,cpu"],
)
cluster.adapt(minimum=0, maximum=200)
client = Client(cluster)

In [4]:
# Submitting jobs
futures = []
for i in range(len(calibrate)):           
    for seed in range(ntrial):
            f = client.submit(file.main,
                              seed,
                              n,ntest,d,noise,
                              calibrate = calibrate[i],
                              sample_split = sample_split[i],
                              marginal_loss = marginal_loss[i],
                              retrain_hypers = retrain_hypers[i],
                              kernel = kernel,
                              niter = niter
                             )
            futures += [f]

In [5]:
# Check on futures
futures

[<Future: pending, key: main-dbb239aa3abc7ed7d511e5dddd916777>,
 <Future: pending, key: main-5ac7628e329b8bf54fb8321687aef582>,
 <Future: pending, key: main-13f736981de92a614be792dd0d33150c>,
 <Future: pending, key: main-91c349efaf14d291ff7551e7e333c011>,
 <Future: pending, key: main-2bda58e6e39ff9b3d6f951ad36aeeab1>,
 <Future: pending, key: main-d3d9944fede7225c33e5299d9d2553d1>,
 <Future: pending, key: main-42c503248551c6fdbe2ca950c5b659f5>,
 <Future: pending, key: main-de7f25c2b132269982a9f4d318b5b821>,
 <Future: pending, key: main-d8df69e7db91bcf38144a3830e0781fd>,
 <Future: pending, key: main-2be654c18f032b4dd9a8101d28c87d34>,
 <Future: pending, key: main-0a046dc54a0ccffc3df2303cca47e8d6>,
 <Future: pending, key: main-7a54f9259a5fd48eeddb7491e255e587>,
 <Future: pending, key: main-7be88593bb0e04c84606dc9b0f51c5e8>,
 <Future: pending, key: main-5a01a452ab11dda67419a15cf5f62aea>,
 <Future: pending, key: main-c2088e81850855ff15a0ea44000a76ff>,
 <Future: pending, key: main-64bc93a3187

In [6]:
# Getting results
results = client.gather(futures)

In [7]:
# Closing client
client.close()
cluster.close()

In [8]:
# Saving results
torch.save(obj = results,
           f = "toy_impspec_ntrial={0}_n={1}_d={2}_noise={3}_kernel={4}.pt".format(ntrial,
                                                                                           n,
                                                                                           d,
                                                                                           noise,
                                                                                           kernel))